# Using Project Debater services for analyzing and finding insights in the survey data 
When you have a large collection of texts representing people’s opinions (such as product reviews, survey answers or  social media), it is difficult to understand the key issues that come up in the data. Going over thousands of comments is prohibitively expensive.  Existing automated approaches are often limited to identifying recurring phrases or concepts and the overall sentiment toward them, but do not provide detailed or actionable insights.

In this tutorial you will gain hands-on experience in using Project Debater services for analyzing and deriving insights from open-ended answers.  

The data we will use is a community survey conducted in the city of Austin in the years 2016 and 2017 (https://data.world/cityofaustin/mf9f-kvkk). In this survey, the citizens of Austin where asked "If there was ONE thing you could share with the Mayor regarding the City of Austin (any comment, suggestion, etc.), what would it be?". 

We will analyze their open-ended answers in different ways by using four Debater services, the *Argument Quality* service, the *Key Point Analysis (KPA)* service, the *Term Wikifier* service and the *Term Relater* service, and we will see how they can be combined into a powerful text analysis tool.

## 1. Run *Key Point Analysis* on 1000 randomly selected sentences from 2016 survey

### 1.1 Read random sample of 1000 sentences from 2016 comments
Let's take a look at the first 5 lines in the *dataset_austin_sentences.csv* file, which holds the Austin survey dataset.

In [1]:
file = open('./dataset_austin_sentences.csv', 'r')
lines = file.readlines()
print('\n'.join(lines[:5]))

id,text,district,year

1,"Dissatisfied traffic and with traffic, timing of street lights.",7,2016

2,EXTREMELY dissatisfied with cit govt.,7,2016

3,"interfering in local businesses (Uber/Lyft, income property owners).",7,2016

4,"Also, extremely dissatisfied with all the free handouts to people who are perfectly capable of earning their own money.",7,2016



The file has all the survey answers after they were split into sentences. Each row in the file corresponds to a single sentence. Each row has the following attributes: \['id', 'text', 'district','year'\]. We will first read the attached csv file into the 'sentences' variable. 

In [2]:
import csv
import random


with open('./dataset_austin_sentences.csv') as csv_file:
    reader = csv.DictReader(csv_file)
    sentences = list(reader)

Let's have a look at the content *sentences* variable. 

In [3]:
print('There are %d sentences in the dataset' % len(sentences))
print('Each sentence is a dictionary with the following keys: %s' % str(sentences[0].keys()))

There are 6274 sentences in the dataset
Each sentence is a dictionary with the following keys: odict_keys(['id', 'text', 'district', 'year'])


Let's select only the sentences from the 2016 survey and randomly sample 1000 out of them. The *Key Point Analysis* service is able to run over hundreds of thousands of sentences, however since the computation is heavy in resources (particularly GPUs) the trial version is limited to 1000 sentences. Using a random.seed(0) is important since we already prepared a hot-cache over these sentences for a quicker *Key Point Analysis* run.

In [4]:
sentences_2016 = [sentence for sentence in sentences if sentence['year'] == '2016']
print('There are %d sentences in the 2016 survey' % len(sentences_2016))
random.seed(0)
random_sample_sentences_2016 = random.sample(sentences_2016, 1000)

There are 3005 sentences in the 2016 survey


### 1.2 Run *Key Point Analysis* on the random sample

Key point analysis is a novel and promising approach for summarization, with an important quantitative angle. This service summarizes a collection of comments on a given topic as a small set of key points. The salience of each key point is given by the number of its matching sentences in the given comments.

Before running the *Key Point Analysis* service we first need to initialize our client.  The DebaterApi object supplies the clients for the various Debater services.   The clients print information using the logger and a suitable verbosity level is should be set. The DebaterApi object is configured with an API key. It should be  retrieved from the Project Debater Early Access Program site.  In this case it is passed by the enviroment variable *DEBATER_API_KEY*.  We then obtain the keypoint client from the DebaterAPI object.

The *Key Point Analysis* service stores the data (and results cache) in a domain. A user can create several domains, one for each dataset. Domains are only accessible to the user who created them.  In this tutorial, we will run all *Key Point Analysis* jobs in the same domain named 'austin_demo'.

Full documentation of the *Key Point Analysis* service can be found [here](https://early-access-program.debater.res.ibm.com/docs/services/keypoints/keypoints_pydoc.html).


In [5]:
from debater_python_api.api.debater_api import DebaterApi
from austin_utils import init_logger
import os

init_logger()
api_key = os.environ['DEBATER_API_KEY']
debater_api = DebaterApi(apikey=api_key)
keypoints_client = debater_api.get_keypoints_client()
domain = 'austin_demo'

Exercise 1:

Let's define a method named *run_kpa*. The method receives a list of sentences (each sentence is a dictionary with the following keys: 'id','text') and runs *Key Point Analysis* on these sentences. The method also receives the *run_params* parameter, which enable us to customize and affect the *Key Point Analysis* operation.

In order to run *Key Point Analysis*, we need to:

1. Upload the comments into a domain using the **keypoints_client.upload_comments(domain=domain, comments_ids=sentences_ids, comments_texts=sentences_texts, dont_split=True)** method. This method receives the domain, a list of comment_ids and a list of comment_texts. By default, when uploading comments into a domain, the *Key Point Analysis* service splits the comments into sentences by default and runs a minor cleansing on the sentences. Since we already splitted the comments into sentences ourselves and we want to *Key Point Analysis* service to use them as is, we will set the *dont_split* parameter to True.

2. Wait till all comments in the domain are processed using the **keypoints_client.wait_till_all_comments_are_processed(domain=domain)** method.

3. Start a *Key Point Analysis* job using the **future = keypoints_client.start_kp_analysis_job(domain=domain, comments_ids=sentences_ids, run_params=run_params)** method. This method receives the domain, a list of comment_ids and a *run_params*. The run_params is a dictionary with various parameters for customizing the job. The job runs in an async manner therefore the method returns a future object.

4. Use the returned future and wait till results are available using the **kpa_result = future.get_result(high_verbosity=True, polling_timout_secs=5)** method. The method waits for the job to finish and eventually returns the result. The result is a dictionary containing the key points (sorted descendingly according to number of matched sentences) and for each key point has a list of matched sentences (sorted descendingly according to their match score). An additional 'none' key point is added which holds all the sentences that don't match any key point.

Our run_kpa method will return this result dictionary. It will also return the unique identifirt for this analysis called *job_id* retreived from the future. We will need this job_id in a following exercise.

In [6]:
def run_kpa(sentences, run_params):
    sentences_texts = [sentence['text'] for sentence in sentences]
    sentences_ids = [sentence['id'] for sentence in sentences]

    keypoints_client.upload_comments(domain=domain, 
                                     comments_ids=sentences_ids, 
                                     comments_texts=sentences_texts, 
                                     dont_split=True)

    keypoints_client.wait_till_all_comments_are_processed(domain=domain)

    future = keypoints_client.start_kp_analysis_job(domain=domain, 
                                                    comments_ids=sentences_ids, 
                                                    run_params=run_params)

    kpa_result = future.get_result(high_verbosity=True, 
                                   polling_timout_secs=5)
    
    return kpa_result, future.get_job_id()

We will now use the method you implemented and run over the random sample and print the result. In order to limit the number of key points in the result to 20, we will use *run_params={'n_top_kps': 20}*.

In [7]:
from austin_utils import print_results

kpa_result_random_1000_2016, _ = run_kpa(random_sample_sentences_2016, {'n_top_kps': 20})
print_results(kpa_result_random_1000_2016, n_sentences_per_kp=2, title='Random sample 2016')

2021-06-22 17:09:25,100 [INFO] keypoints_client.py 284: uploading 1000 comments in batches
2021-06-22 17:09:25,101 [INFO] keypoints_client.py 214: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:09:26,149 [INFO] keypoints_client.py 301: uploaded 1000 comments, out of 1000
2021-06-22 17:09:26,150 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:09:26,861 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 1000, 'processed_sentences': 3122}
2021-06-22 17:09:36,866 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:09:37,488 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 0, 'processed_sentences': 3122}
2021-06-22 17:09:37,48

Random sample 2016 coverage: 28.36
Random sample 2016 key points:
39 - We need better mass transit!
	- Need more bus routes!
	- NEED BETTER PUBLIC TRANSPORTATION
34 - Affordable housing, traffic, cleaner streets/roads.
	- TRAFFIC AND AFFORDABLE HOUSING ARE THE BIGGEST PROBLEM TO LIVING HERE.
	- Also, the traffic in Austin is ridiculous and the lack of public transportation needs
	  improvements.
24 - Affordable housing is very important.
	- Affordable housing is crucial, & keeping seniors in their homes is part of that challenge!
	- Affordable housing MUST become a reality/ahora!
21 - Homeowners taxes are a problem.
	- Cost of living here is to high & tax for my home is to high.
	- High property taxes seem to be one of the two reasons why people I know are leaving
	  Austin (traffic being the other reason).
19 - Water costs too much.
	- The cost or water is excessive and way too high.
	- Also, the cost of my water bill is insanely high and I am about to protest it!
17 - Provide public 

## 2. Run *Key Point Analysis* on 1000 top quality sentences from 2016 survey
### 2.1 Select top 1000 sentences from 2016 data using the *Argument Quality* service
The answers in the Austin Survey dataset vary in length, style and quality. Selecting the sentences randomly may lead to running over many sentences that are not very informative. Running over the randomly selected sentences reached a 28.36% coverage. This means that only 28.36% of the sentences matched a key point. In order to improve the coverage and the quality of our results, we will now run over higher quality sentences and select the 1000 sentences with the highest *Argument Quality* score. The *Argument Quality* service receives pairs of \[sentence, topic\] and returns a score indicating whether the sentence is phrased in grammatically correct, clear and concise language.   The ranking of the quality is based on the machine learning model, which was trained on human assesments of over 30,000 arguments. 

In [8]:
from austin_utils import print_top_and_bottom_k_sentences

def get_top_quality_sentences(sentences, top_k, topic):    
    arg_quality_client = debater_api.get_argument_quality_client()
    sentences_topic = [{'sentence': sentence['text'], 'topic': topic} for sentence in sentences]
    arg_quality_scores = arg_quality_client.run(sentences_topic)
    sentences_and_scores = zip(sentences, arg_quality_scores)
    sentences_and_scores_sorted = sorted(sentences_and_scores, key=lambda x: x[1], reverse=True)
    sentences_sorted = [sentence for sentence, _ in sentences_and_scores_sorted]
    print_top_and_bottom_k_sentences(sentences_sorted, 10)
    return sentences_sorted[:top_k]

sentences_2016_top_1000_aq = get_top_quality_sentences(sentences_2016, 1000, 'Austin is a great place to live')

ArgumentQualityClient: 100%|█████████▉| 3000/3005 [00:05<00:00, 542.24it/s]2021-06-22 17:10:03,435 [INFO] argument_quality_client.py 24: argument_quality_client.run = 5942.997932434082ms.


Top 10 quality sentences: 
	- Affordable housing is essential to keep Austin diverse, welcoming, and growing in the
	  ways that reflect the progressive ideals of this city and the future generations.
	- Austin has a unique charm, with high quality customer service,  and great quality of
	  life that will quickly deteriorate with this rapid urban sprawl.
	- We need to make sure our city continues to be an example across the country for
	  unwavering progress, with regards to energy, policing, fair housing and employment,
	  non-discrimination policy, and creating a sound infrastructure to accommodate the city's
	  rapid growth.
	- We need a coordinated effort by many agencies to find housing, training, rehab,
	  healthcare and work for our homeless.
	- Affordability and diversity (in culture and age groups) are extremely important to
	  keeping Austin the vibrant, egalitarian city it is.
	- KEEP AUSTIN QUALITY OF LIFE BY INVESTING IN TRAILS, BIKE LANES, SIDEWALKS, PARKS, THESE
	  ARE I

### 2.2 Run *Key Point Analysis* over the selected sentences
We will now run the *run_kpa* method over the top 1000 quality sentences

In [9]:
kpa_result_top_aq_1000_2016, _ = run_kpa(sentences_2016_top_1000_aq, {'n_top_kps': 20})
print_results(kpa_result_top_aq_1000_2016, n_sentences_per_kp=2, title='Top aq 2016')

2021-06-22 17:10:03,446 [INFO] keypoints_client.py 284: uploading 1000 comments in batches
2021-06-22 17:10:03,448 [INFO] keypoints_client.py 214: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:10:04,639 [INFO] keypoints_client.py 301: uploaded 1000 comments, out of 1000
2021-06-22 17:10:04,640 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:10:05,303 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 1000, 'processed_sentences': 3122}
2021-06-22 17:10:15,308 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:10:15,884 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 0, 'processed_sentences': 3122}
2021-06-22 17:10:15,88

Top aq 2016 coverage: 41.05
Top aq 2016 key points:
88 - Traffic congestion needs major improvement
	- Austin need improved transportation infrastructure to alleviate current traffic and
	  accommodate rapid population growth.
	- Fast population growth is the cities biggest problem in areas such as congestion and
	  expensiveness.
78 - We need affordable housing.
	- Affordable housing is crucial, & keeping seniors in their homes is part of that challenge!
	- We need better housing solutions for low income and homeless individuals.
42 - Integrated transportation is critical.
	- a public transit train system is a necessity
	- Steiner Ranch area needs more access to public transportation (bus, train, etc>) and
	  taxi/ridesharing  to/from downtown and airport
30 - UTILITY RATES & PROPERTY TAXES ARE OUTRAGEOUS
	- PROPERTY TAXES ARE TOO HIGH.
	- PROPERTY TAXES ARE OUT OF CONTROL.
20 - THE HOMELESS POPULATION NEEDS MORE HELP/ATTENTION
	- HOMELESS SITUATION NEEDS TO BE ADDRESSED IN THE DOWNTO

### 2.3 Customize key point analysis
It is possible to costumize and affect the analysis by passing different parameters in the *run_params* dictionary. In this subsection we will see few examples.

#### 2.3.1 Hierarchical key points
Often, few key points address a similar topic. In order to get an even clearer sumamry of the data, we can group similar key points together using the *Hierarchical Key Points* feature. To acrivate it we add two additional parameters to run_params: {'perform_kp_hierarchy': True, 'kp_hierarchy_threshold': 0.3}. *perform_kp_hierarchy* activates the feature and *kp_hierarchy_threshold* sets a threshold for grouping similar key points. The lower the threshold, more key points are grouped with lower similarity.

In [10]:
kpa_result_top_aq_1000_2016, _ = run_kpa(sentences_2016_top_1000_aq, 
                                    {'n_top_kps': 20, 'perform_kp_hierarchy': True, 'kp_hierarchy_threshold': 0.3})
print_results(kpa_result_top_aq_1000_2016, n_sentences_per_kp=2, title='Top aq 2016, hierarchical')

2021-06-22 17:10:41,094 [INFO] keypoints_client.py 284: uploading 1000 comments in batches
2021-06-22 17:10:41,096 [INFO] keypoints_client.py 214: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:10:42,215 [INFO] keypoints_client.py 301: uploaded 1000 comments, out of 1000
2021-06-22 17:10:42,216 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:10:42,932 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 1000, 'processed_sentences': 3122}
2021-06-22 17:10:52,924 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:10:53,472 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 0, 'processed_sentences': 3122}
2021-06-22 17:10:53,47

Top aq 2016, hierarchical coverage: 41.05
Top aq 2016, hierarchical key points:
107 - 88 - Traffic congestion needs major improvement
	- Austin need improved transportation infrastructure to alleviate current traffic and
	  accommodate rapid population growth.
	- Fast population growth is the cities biggest problem in areas such as congestion and
	  expensiveness.
	15 - There is constant traffic and noise pollution.
		- IH35 & MoPac congestion is negative for local economy and causes air pollution and
		  causes safety hazards, i.e. dangerous to drive here.
		- My neighborhood has been subjected to 2+ years of noise, dirt, filthy side walks,
		  overgrown common areas, rude drivers, etc.
	4 - Roads are poorly maintain.
		- Major thoroughfares poorly planned, not functioning, city streets need repair.
		- This lack of maintenance makes Austin look run down.
93 - 78 - We need affordable housing.
	- Affordable housing is crucial, & keeping seniors in their homes is part of that challenge!

#### 2.3.2 Increase coverage by decreasing the matching threshold
Running over higher quality sentences we managed to increase our coverage to 41.05%. In order to increase the coverage more, we will add another parameter to the run_params called *mapping_threshold*. 

The mapping_threshold is responsible of deciding whether a sentences matches (supports) a key point. Therefore reducing the threshold from the 0.99 default value makes more sentences match key points and increases the coverage, at the risk of reducing the precision.

In [11]:
kpa_result_top_aq_1000_2016, kpa_top_aq_1000_2016_job_id = run_kpa(sentences_2016_top_1000_aq, 
                                                                {'n_top_kps': 20, 'mapping_threshold': 0.95})
print_results(kpa_result_top_aq_1000_2016, n_sentences_per_kp=2, title='Top aq 2016')

2021-06-22 17:11:24,257 [INFO] keypoints_client.py 284: uploading 1000 comments in batches
2021-06-22 17:11:24,259 [INFO] keypoints_client.py 214: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:11:25,443 [INFO] keypoints_client.py 301: uploaded 1000 comments, out of 1000
2021-06-22 17:11:25,444 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:11:26,140 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 1000, 'processed_sentences': 3122}
2021-06-22 17:11:36,145 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:11:36,716 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 0, 'processed_sentences': 3122}
2021-06-22 17:11:36,71

Top aq 2016 coverage: 49.89
Top aq 2016 key points:
93 - Traffic congestion needs major improvement
	- Austin need improved transportation infrastructure to alleviate current traffic and
	  accommodate rapid population growth.
	- Fast population growth is the cities biggest problem in areas such as congestion and
	  expensiveness.
91 - We need affordable housing.
	- Affordable housing is crucial, & keeping seniors in their homes is part of that challenge!
	- We need better housing solutions for low income and homeless individuals.
53 - Integrated transportation is critical.
	- a public transit train system is a necessity
	- Steiner Ranch area needs more access to public transportation (bus, train, etc>) and
	  taxi/ridesharing  to/from downtown and airport
34 - UTILITY RATES & PROPERTY TAXES ARE OUTRAGEOUS
	- PROPERTY TAXES ARE TOO HIGH.
	- PROPERTY TAXES ARE OUT OF CONTROL.
26 - South Austin is losing businesses and residents.
	- Slow down the commercial/entertainment oriented develop

The coverage was indeed increased to about 50%. Let's examine the bottom 5 sentences that were matched to the first key point and make sure that the precision is still high.

In [12]:
from austin_utils import print_bottom_matches_for_kp
print_bottom_matches_for_kp(kpa_result_top_aq_1000_2016, 'Traffic congestion needs major improvement', 5)


Bottom 5 matches:
	- The biggest problem Austin has is a lack of viable north-south arteries.
	- If the train traveled to more locations there would be more people on the train and less
	  traffic.
	- Please plan for the future through investments in a comprehensive transit plan rather
	  than quick fixes related to traffic.
	- Providing more bike lanes and dashed lanes is adding to the problem because drivers have
	  no idea what the rules are, and neither do bicycles.
	- Traffic in Austin is a major problem no matter which road you use.


## 3. Run *Key Point Analysis* over 2017 survey using the key points from 2016 survey
### 3.1 Select top 1000 sentences from 2017 data using the *Argument Quality* service
It is very useful to be able to compare between different subsets of the data (compare between different years, different districts, etc'). We will demonstrate how easy it is to compare the 2017 data to the 2016 data. A similar comparisson can be done between districts or other subsets. 

Let's first filter the 2017 sentences and take the top 1000 quality sentences, as done for the 2016 sentences.

In [13]:
sentences_2017 = [sentence for sentence in sentences if sentence['year'] == '2017']
sentences_2017_top_1000_aq = get_top_quality_sentences(sentences_2017, 1000, 'Austin is a great place to live')

ArgumentQualityClient: 100%|██████████| 3269/3269 [00:06<00:00, 527.65it/s]

Top 10 quality sentences: 
	- Affordable housing is important to promote diversity in Austin.
	- More space (and more affordable, quality housing) means more artists, more artwork of
	  all mediums, and a more culturally diverse and enriched city.
	- AUSTIN NEEDS A BETTER PUBLIC TRANSPORTATION SYSTEM THAT CAN ACCOMMODATE MORE PEOPLE TO
	  HELP WITH TRAFFIC AND OVERALL GROWTH OF THE CITY.
	- We are at a tipping point in Austin, with such amazing growth from new businesses, that
	  we have a wonderful opportunity to address basic needs.
	- The middle and lower income people need more affordable housing; property taxes need to
	  be lowered and be more equitable.
	- Protect historic neighborhoods, they offer social and economic sustainability.
	- WE NEED TO KEEP AUSTIN AFFORDABLE FOR ALL SO WE CAN BENEFIT FROM THAT DIVERSITY.
	- THE QUALITY OF LIFE IN AUSTIN IS DEGRADING BECAUSE OF THE LACK OF COMPREHENSIVE PLAN TO
	  ADDRESS THE HOMELESS EPIDEMIC TO FIX TRAFFIC/ROAD CONGESTION AND THE LA

### 3.2 Run *Key Point Analysis* over top 1000 quality 2017 sentences using the key points from 2016
Exercise 2:<br/>
In order to compare the 2017 sentences to 2016 sentences we will want to map the 2017 sentences to the same key points extracted on the 2016 sentences (otherwise different key points could be automattically extracted on the 2017 sentences and it would be hard to compare between them).

For this end we will reimplement the *run_kpa* method (please copy paste the previous one and modify it). This time the method will receive a new *key_points_by_job_id* parameter. This parameter is passed to the *key_points_by_job_id* parameter in the **future = keypoints_client.start_kp_analysis_job(domain=domain, comments_ids=sentences_ids, run_params=run_params, key_points_by_job_id=key_points_by_job_id)** method. When *None* is passed to *key_points_by_job_id*, key points are automatically extracted, however when it is set with a *job_id* of a previous job it uses the key points from that job and matches all sentences to them.

In [14]:
def run_kpa(sentences, run_params, key_points_by_job_id=None):
    sentences_texts = [sentence['text'] for sentence in sentences]
    sentences_ids = [sentence['id'] for sentence in sentences]

    keypoints_client.upload_comments(domain=domain,
                                     comments_ids=sentences_ids,
                                     comments_texts=sentences_texts,
                                     dont_split=True)

    keypoints_client.wait_till_all_comments_are_processed(domain=domain)

    future = keypoints_client.start_kp_analysis_job(domain=domain, comments_ids=sentences_ids,
                                                    run_params=run_params,
                                                    key_points_by_job_id=key_points_by_job_id)

    kpa_result = future.get_result(high_verbosity=True, polling_timout_secs=5)
    
    return kpa_result, future.get_job_id()

Let's use the new *run_kpa* and provide it with the *top 1000 quality sentences from 2017* and the job_id of *top 1000 quality sentences from 2016*.

In [15]:
kpa_result_top_aq_1000_2017, _ = run_kpa(sentences_2017_top_1000_aq, 
                                    {'n_top_kps': 20, 'mapping_threshold': 0.95}, kpa_top_aq_1000_2016_job_id)
print_results(kpa_result_top_aq_1000_2017, n_sentences_per_kp=2, title='Top aq 2017, using 2016 key points')

2021-06-22 17:12:08,658 [INFO] keypoints_client.py 284: uploading 1000 comments in batches
2021-06-22 17:12:08,659 [INFO] keypoints_client.py 214: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:12:09,905 [INFO] keypoints_client.py 301: uploaded 1000 comments, out of 1000
2021-06-22 17:12:09,906 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:12:10,607 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 1000, 'processed_sentences': 3122}
2021-06-22 17:12:20,613 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:12:21,224 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 0, 'processed_sentences': 3122}
2021-06-22 17:12:21,22

Top aq 2017, using 2016 key points coverage: 49.15
Top aq 2017, using 2016 key points key points:
122 - We need affordable housing.
	- Please consider working towards more affordable quality housing for middle and lower
	  income families in the City of Austin.
	- And try to increase housing affordability, especially for seniors and low-income families.
93 - Traffic congestion needs major improvement
	- HWY & ROAD SYSTEMS NEED IMPROVEMENT, THE FLOW OF TRAFFIC NEEDS TO MAKE SENSE, WE NEED
	  MORE READILY AVAILABLE PUBLIC TRANSPORTATION SYSTEMS
	- The rush hour traffic on city highways and major city streets needs significant
	  improvement.
43 - Integrated transportation is critical.
	- Austin needs better public transportation
	- WE NEED A BETTER TRANSPORTATION SYSTEM - A COMBINATION OF TRAINS AND BUSES.
39 - UTILITY RATES & PROPERTY TAXES ARE OUTRAGEOUS
	- PROPERTY TAXES ARE WAY TOO HIGH.
	- Wastewater service fees and property taxes are ridiculously high.
33 - THE HOMELESS POPULATION

Since both jobs have the same key points, we can now easily compare the two results.

In [16]:
from austin_utils import compare_results

compare_results(kpa_result_top_aq_1000_2016, '2016', kpa_result_top_aq_1000_2017, '2017')

2016 - 2017 comparison:
+-----------------------------------------------------------------------+------+------+--------+
|                               key point                               | 2016 | 2017 | change |
+-----------------------------------------------------------------------+------+------+--------+
|               Traffic congestion needs major improvement              |  93  |  93  |   0%   |
|                      We need affordable housing.                      |  91  | 122  |  34%   |
|                 Integrated transportation is critical.                |  53  |  43  |  -19%  |
|             UTILITY RATES & PROPERTY TAXES ARE OUTRAGEOUS             |  34  |  39  |  14%   |
|            South Austin is losing businesses and residents.           |  26  |  16  |  -39%  |
|           THE HOMELESS POPULATION NEEDS MORE HELP/ATTENTION           |  23  |  33  |  43%   |
|                   HOUSING PRICES ARE OUT OF CONTROL.                  |  18  |  20  |  11%   |
| Aust

Note: This comparision is for illustration only. Given that we ran on a subset of comments, the statistical significant of difference between the years is limited, except for the most recurring keypoints.

## 4. Deep dive into the *traffic problem* in Austin using the *Term Wikifier* and *Term Relater* services
As we've seen in the 2016 results, that the traffic problem in Austin is significant. In this section we will use the *Term Wikifier* and *Term Relater* services to select a subset of the sentences related to the *Traffic* topic and run *Key Point Analysis* over them. 

The *Term Wikifier* service runs over sentences and identifies the Wikipedia concepts that are referenced by phrases in the sentence text.  Concepts correspond to Wikipedia articles.  Each occurance of a concept in the sentence is called a *mention*.  For example, the sentence "My car insurance went up 20% due to vehicle thefts and burglary" mentions three Wikipedia concepts: The phrase "car insurance" is mapped to the concept *Vehicle insurance*; the phrase "vehicle thefts" is mapped to the concept *Motor vehicle theft* and the phrase "burglary" is mapped to the concept *Burglary*.

The *Term Relater* service runs over pairs of Wikipedia concepts and scores how closely these concepts are related.  For example, the *Car* concept is very related to the *Traffic* concept but the *Cat* concept is not very related to the *Traffic* concept.

We will use the *Term Wikifier* to extract all mentions in all sentences; then use the *Term Relater* to select a subset of these mentions which are related to the 'Traffic' concept; then select all sentences that have mentions related to the 'Traffic' concept; and finally run *Key Point Analysis* over them. Running over these sentences will create key points specifically to the traffic problem in Austin and expose insights and suggestions related to it.

### 4.1 Calculate the mentions in the sentences using the *Term Wikifier*
Exercise 3:

Please complete the missing parts in the *get_sentence_to_mentions(sentences_texts)* method. The method uses the *Term Wikifier* service, calculates the mentions for each sentence and stores it in a dictionary named *sentence_to_mentions*. 

The *Term Wikifier* client runs over the sentences_texts using the **mentions_list = term_wikifier_client.run(sentences_texts)** method and returns a list of mentions_lists.

In [17]:
def get_sentence_to_mentions(sentences_texts):
    term_wikifier_client = debater_api.get_term_wikifier_client()

    mentions_list = term_wikifier_client.run(sentences_texts)
    
    sentence_to_mentions = {}
    for sentence_text, mentions in zip(sentences_texts, mentions_list):
        sentence_to_mentions[sentence_text] = set([mention['concept']['title'] for mention in mentions])
    return sentence_to_mentions

Let's calculate the mentions on all 2016 sentences"

In [18]:
sentences_2016_texts = [sentence['text'] for sentence in sentences_2016]
sentence_to_mentions = get_sentence_to_mentions(sentences_2016_texts)

TermWikifierClient: 100%|█████████▉| 3000/3005 [00:12<00:00, 243.73it/s]2021-06-22 17:12:42,054 [INFO] term_wikifier_client.py 34: term_wikifier_client.run = 12746.110916137695ms.


### 4.2 Find the mentions that relate to the *traffic* concept using the *Term Relater* service
Since we're interested in the *Traffic* concept, we will now take all mentions and find the ones that are related to that concept. Then we will select all sentences that have at least one mention that is related to the *Traffic* concept.

In [19]:
all_mentions = set([mention for sentence in sentence_to_mentions 
                   for mention in sentence_to_mentions[sentence]])

Exercise 4:<br/>
Please complete the missing parts in the *get_related_mentions(concept, threshold, all_mentions)* method. It receives a given concept, a threshold and all_mentions. It then uses the *Term Relater* service to calculate the relatedness between the mentions and the concept and returns all mentions that have relatedness score above the given threhold. The *term_relater_client* runs over the pairs using the **scores = term_relater_client.run(concept_mention_pairs)** method and returns a list of scores.

In [20]:
def get_related_mentions(concept, threshold, all_mentions):
    term_relater_client = debater_api.get_term_relater_client()
    concept_mention_pairs = [[concept, mention] for mention in all_mentions]

    scores = term_relater_client.run(concept_mention_pairs)
    
    return [mention for mention, score in zip(all_mentions, scores) if score > threshold]

We will now use the method you've implemented and find the mentions that match the *traffic* concept.

In [21]:
matched_mentions = get_related_mentions('Traffic', 0.5, all_mentions)
print(matched_mentions)

TermRelaterClient: 100%|██████████| 1634/1634 [00:02<00:00, 526.68it/s]2021-06-22 17:12:44,920 [INFO] term_relater_client.py 37: term_relater_client.run = 2813.074827194214ms.


['Frontage road', 'Motor vehicle', 'Driving under the influence', 'Driving', 'Public transport timetable', 'Stop sign', 'Bicycle safety', 'Mode of transport', 'Truck', 'Police transport', 'Single-vehicle crash', 'Capital Metropolitan Transportation Authority', 'Transport network', 'Traffic flow', 'Thoroughfare', 'Traffic congestion', 'State highway', 'Car', 'Bus station', 'Traffic light', 'Traffic collision', 'Public transport bus service', 'Transport', 'Gridlock', 'Travel', 'Highway', 'Ridesharing company', 'Vehicle insurance', 'Bus driver', 'Road surface', 'Cycling', 'Bicycle', 'Park and ride', 'One-way traffic', 'Controlled-access highway', 'Highway systems by country', 'Rail freight transport', 'Pedestrian', 'Bus', 'Boulevard', 'Parking', 'Right of way', 'Pedestrian facilities', 'Speed bump', 'Traffic management', 'Signal', 'Transit bus', 'Parking violation', 'Vehicle', 'Bus lane', 'Bridge', 'Driveway', 'Parkway', 'Traffic ticket', 'Rail transport', 'Road', 'Road traffic safety', '

### 4.3 Run *Key Point Analysis* over the sentences that relate to the *Traffic* concept
Let's select the sentences that have mentions that are related to the *Traffic* concept and run over them. We will need to switch back from sentences_texts to sentences dictionaries since our *run_kpa* method needs the sentences dictionaries.

In [22]:
matched_sentences_texts = [sentence for sentence in sentences_2016_texts 
                     if len(sentence_to_mentions[sentence].intersection(matched_mentions)) > 0]
matched_sentences = [sentence for sentence in sentences_2016 if sentence['text'] in matched_sentences_texts]
matched_sentences = matched_sentences if len(matched_sentences) <= 1000 else random.sample(matched_sentences, 1000)
print('Running over %d sentences' % len(matched_sentences))

Running over 980 sentences


Finally, let's run over these sentences and examine the *Traffic* related key points

In [23]:
kpa_result_traffic_2016, _ = run_kpa(matched_sentences, {'n_top_kps': 20, 'mapping_threshold': 0.99}, None)
print_results(kpa_result_traffic_2016, n_sentences_per_kp=2, title='Traffic KPA 2016')

2021-06-22 17:12:44,983 [INFO] keypoints_client.py 284: uploading 980 comments in batches
2021-06-22 17:12:44,984 [INFO] keypoints_client.py 214: client calls service (post): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:12:45,961 [INFO] keypoints_client.py 301: uploaded 980 comments, out of 980
2021-06-22 17:12:45,962 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:12:46,660 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 980, 'processed_sentences': 3122}
2021-06-22 17:12:56,665 [INFO] keypoints_client.py 214: client calls service (get): https://keypoint-matching-backend.debater.res.ibm.com/comments
2021-06-22 17:12:57,225 [INFO] keypoints_client.py 313: domain: austin_demo, comments status: {'processed_comments': 3122, 'pending_comments': 0, 'processed_sentences': 3122}
2021-06-22 17:12:57,226 [I

Traffic KPA 2016 coverage: 62.17
Traffic KPA 2016 key points:
174 - PLEASE ADDRESS THE TRAFFIC CONGESTION
	- Pay closer attention to the Four Points area - extreme traffic congestion that is only
	  getting worse!
	- Austin traffic (especially the morning commute) needs help!
74 - We need better mass transit!
	- WE NEED MASS TRANSIT TO THE AIRPORT!
	- Steiner Ranch area needs more access to public transportation (bus, train, etc>) and
	  taxi/ridesharing  to/from downtown and airport
49 - Fix traffic congestion - better planning needed.
	- Need to plan for the growth and traffic this city is now experiencing much better.
	- NEED BETTER TRAFFIC FLOW PLANNING.
45 - Affordable housing, traffic, cleaner streets/roads.
	- Ensure good traffic flow, safety, affordable housing, city planning, and cleanliness.
	- WE NEED AFFORDABLE HOUSING, BETTER ROADS AND MORE ROADS
42 - Transportation improvements--light rail, bike facilities, roadways, sidewalks
	- Austin needs to get serious about alternat

### 4.4 Conclusion

In this tutorial, we showed how *Key Point Analysis* can provide you with detailed insights over survey data right out of the box - significantly reducing the effort required by a data scientist to analyze the data.  We also demonstrated how key point analysis over unstructured text can be combined with available structured information, to provide new views over the data.   Finally, we showed how utilizing of additional Project Debater text analysis services such as *Argument Quality*, *Term Wikifier* , and *Term Relater* can further improve the quality of the results.